In [7]:
import json
from pathlib import Path

def load_frames(json_path):
    """Load components→frames JSON."""
    return json.loads(Path(json_path).read_text())

def get_frame(data, comp_idx, frame_idx):
    """Retrieve the frame dict for given component/frame."""
    return data[str(comp_idx)]["frames"][str(frame_idx)]

def normalize_nodes(raw_nodes):
    """
    Convert list of nodes (each a list of labels) into a set of tuples.
    E.g. ['A:ALA:4','A:ALA:6'] → ('A:ALA:4','A:ALA:6').
    """
    return set(tuple(node) for node in raw_nodes)

def normalize_edges(raw_edges):
    """
    Convert raw edges (each [node1_list, node2_list]) into a set of 
    frozensets of two node‐tuples: frozenset({tuple(node1),tuple(node2)}).
    """
    out = set()
    for edge in raw_edges:
        if len(edge) != 2:
            continue
        t1 = tuple(edge[0])
        t2 = tuple(edge[1])
        out.add(frozenset((t1, t2)))
    return out

def frames_equal(f1, f2):
    """True if nodes and edges match exactly (order‐independent)."""
    n1 = normalize_nodes(f1.get("nodes", []))
    n2 = normalize_nodes(f2.get("nodes", []))
    e1 = normalize_edges(f1.get("edges", []))
    e2 = normalize_edges(f2.get("edges", []))
    return (n1 == n2) and (e1 == e2)

def is_subset(f_small, f_large):
    """
    True if every node‐tuple in f_small is in f_large and
    every edge (as frozenset of two node‐tuples) in f_small is in f_large.
    """
    ns = normalize_nodes(f_small.get("nodes", []))
    nl = normalize_nodes(f_large.get("nodes", []))
    es = normalize_edges(f_small.get("edges", []))
    el = normalize_edges(f_large.get("edges", []))
    return ns.issubset(nl) and es.issubset(el)

def compare_frames(data, c1, f1, c2, f2):
    """Pretty‐print equal / subset relations between two frames."""
    A = get_frame(data, c1, f1)
    B = get_frame(data, c2, f2)
    print(f"Comparing comp{c1}_frame{f1} vs comp{c2}_frame{f2}")
    print(f"  Equal?           {frames_equal(A, B)}")
    print(f"  Frame1 ⊆ Frame2? {is_subset(A, B)}")
    print(f"  Frame2 ⊆ Frame1? {is_subset(B, A)}")

# --- USAGE in a Jupyter cell ---

# 1. Load your JSON (adjust path!)
data = load_frames("data/All/graph_All.json")

# 2. Compare a pair
compare_frames(data, 2, 1, 2, 2)

# 3. (Optional) Create a full pairwise DataFrame
import pandas as pd

rows = []
for c1, comp in data.items():
    for f1 in comp["frames"]:
        for c2, comp2 in data.items():
            for f2 in comp2["frames"]:
                A = get_frame(data, int(c1), int(f1))
                B = get_frame(data, int(c2), int(f2))
                rows.append({
                    "c1": int(c1), "f1": int(f1),
                    "c2": int(c2), "f2": int(f2),
                    "equal": frames_equal(A, B),
                    "subset_1_in_2": is_subset(A, B),
                    "subset_2_in_1": is_subset(B, A)
                })

df = pd.DataFrame(rows)
df.head()


KeyError: '2'